### General setup

In [106]:
%matplotlib widget

## Raw dataset preparation

### Fiftyone set up

Import fiftyone

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

Set download location

In [2]:
import pathlib
datasets_path = pathlib.Path().resolve() / "raw_datasets"
fo.config.dataset_zoo_dir = datasets_path

List all fiftyone datasets

In [14]:
fo.list_datasets()

['bdd100k-train', 'coco-2017-train-10000', 'kitti-train']

### KITTI Dataset

#### Training and Validation

In [51]:
kitti_train = foz.load_zoo_dataset("kitti", split="train")

Split 'train' already downloaded
Loading existing dataset 'kitti-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Extract 20% from training set for validation

In [57]:
index = int(len(kitti_train) * 0.8)
kitti_train_view = kitti_train.shuffle()[:index]
kitti_validation_view = kitti_train.shuffle()[index:]

Check if all images are used and none is duplicated (due to rounding of index)

In [59]:
len(kitti_train_view) + len(kitti_validation_view) == len(kitti_train)

True

In [62]:
kitti_train_6000 = fo.Dataset()
kitti_train_6000.merge_samples(kitti_train_view)
kitti_train_6000.name = 'kitti_train_6000'

#### Test

In [ ]:
kitti_test = foz.load_zoo_dataset("kitti", split="test")

Dataset operations

In [5]:
view = kitti_train.view()

In [ ]:
view.take(len(view)).first()

In [ ]:
x = view.take(10).first()
gt = x.ground_truth
gt.detections[0]
x.filepath

### COCO Dataset

In [49]:
def download_coco(split, max_samples):
    # Restarting download after connection timeout
    try:
        coco_train = fo.zoo.load_zoo_dataset("coco-2017", split=split, label_types=["detections"], classes=["person", "bicycle", "car", "motorcycle", "bus", "train", "truck", 
        "traffic light", "fire hydrant", "parking meter", "bench", "cat", "dog", "chair", "couch", "bed", "dining table", "tv", "laptop"], only_matching=True, max_samples=max_samples)
    except:
        coco_train = download_coco(split, max_samples)

    return coco_train

In [50]:
coco_train = download_coco('train', 12000)

Found annotations at 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\coco-2017\raw\instances_train2017.json'
11798 images found; downloading the remaining 202
 100% |██████████████████| 202/202 [22.0s elapsed, 0s remaining, 10.0 images/s]     
Writing annotations for 12000 downloaded samples to 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\coco-2017\train\labels.json'
Dataset info written to 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\coco-2017\info.json'
Loading 'coco-2017' split 'train'
 100% |█████████████| 12000/12000 [44.4s elapsed, 0s remaining, 228.1 samples/s]      
Dataset 'coco-2017-train-12000' created


In [ ]:
coco_validation = download_coco('validation', 2400)

In [ ]:
coco_test = download_coco('test', 2400)

### BDD Dataset

#### Training

In [4]:
bdd_train = foz.load_zoo_dataset(
    "bdd100k",
    split="train",
    source_dir=r'raw_datasets/bdd100k/',
)

Preparing split 'train' in 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\bdd100k\train'
Preparing training images...
Preparing training labels...
Preparing validation images...
Preparing validation labels...
Preparing test images...
Parsing dataset metadata
Found 70000 samples
Dataset info written to 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\bdd100k\info.json'
Loading 'bdd100k' split 'train'
 100% |█████████████| 70000/70000 [1.0h elapsed, 0s remaining, 42.3 samples/s]      
Dataset 'bdd100k-train' created


Take only 6000 images from shuffled dataset

In [17]:
bdd_train_view = bdd_train.view()
bdd_train_view = bdd_train_view.shuffle()[:6000]

Save view as a new dataset

In [46]:
bdd_train_6000 = fo.Dataset()
bdd_train_6000.merge_samples(bdd_train_view)
bdd_train_6000.name = 'bdd_train_6000'

#### Validation

In [ ]:
bdd_validation = foz.load_zoo_dataset(
    "bdd100k",
    split="validation",
    source_dir=r'raw_datasets/bdd100k/',
)

#### Test

In [ ]:
bdd_test = foz.load_zoo_dataset(
    "bdd100k",
    split="test",
    source_dir=r'raw_datasets/bdd100k/',
)

## TF Record Creation

### Preprocess data

Delete images without labels and containing only "traffic sign" label from BDD Dataset

In [117]:
dropped_ids = []
for img in bdd_train_6000:
    if img.detections:
        det_labels = []
        for det in img.detections.detections:
            det_labels.append(det.label)
        det_labels = set(det_labels)
        if len(det_labels) == 1 and "traffic sign" in det_labels:
            dropped_ids.append(img.id)
    else:
        dropped_ids.append(img.id)

In [ ]:
bdd_train_6000.delete_samples(dropped_ids)
bdd_train_6000.save()

Delete images without labels and with labels containig only "Misc" or "DontCare" from KITTI dataset

In [150]:
correct_labels = ["Car", "Pedestrian", "Van", "Cyclist", "Truck", "Tram", "Person_sitting"]

In [159]:
dropped_ids = []
for img in kitti_train_6000:
    det_labels = []
    for det in img.ground_truth.detections:
        det_labels.append(det.label)
    for label in correct_labels:
        if label in det_labels:
            break
        dropped_ids.append(img.id)

In [155]:
kitti_train_6000.delete_samples(dropped_ids)
kitti_train_6000.save()

### Datasets statistics

In [96]:
import pandas as pd

In [127]:
bdd_labels = []
for img in bdd_train_6000:
    for det in img.detections.detections:
        bdd_labels.append(det.label)

In [126]:
kitti_labels = []
for img in kitti_train_6000:
    for det in img.ground_truth.detections:
        kitti_labels.append(det.label)

In [163]:
coco_labels = []
for img in coco_train:
    for det in img.ground_truth.detections:
        coco_labels.append(det.label)

In [129]:
bdd_labels = pd.DataFrame({"bdd": bdd_labels})

In [138]:
bdd_labels["bdd"].value_counts()

car              62162
traffic sign     20027
traffic light    15479
person            7273
truck             2592
bus               1004
bike               625
rider              376
motor              251
train               21
Name: bdd, dtype: int64

In [131]:
kitti_labels = pd.DataFrame({"kitti": kitti_labels})

In [137]:
kitti_labels["kitti"].value_counts()

Car               22938
DontCare           9007
Pedestrian         3555
Van                2308
Cyclist            1291
Truck               848
Misc                792
Tram                403
Person_sitting      184
Name: kitti, dtype: int64

In [164]:
coco_labels = pd.DataFrame({"coco": coco_labels})

In [165]:
coco_labels["coco"].value_counts()

person           35278
car               6050
chair             5027
dining table      2065
traffic light     1740
truck             1358
bench             1277
motorcycle        1132
bicycle            873
dog                770
bus                768
couch              725
tv                 719
laptop             660
train              635
cat                621
bed                578
fire hydrant       253
parking meter      179
Name: coco, dtype: int64